In [2]:
import kagglehub
import pandas as pd
import numpy as np


In [4]:
import lightgbm as lgb

In [3]:
# path = kagglehub.dataset_download("vinayakshanawad/cement-manufacturing-concrete-dataset")
path = "/data/ephemeral/home/.cache/kagglehub/datasets/vinayakshanawad/cement-manufacturing-concrete-dataset/versions/1"
df = pd.read_csv(path + "/concrete.csv")

df.head()

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength
0,141.3,212.0,0.0,203.5,0.0,971.8,748.5,28,29.89
1,168.9,42.2,124.3,158.3,10.8,1080.8,796.2,14,23.51
2,250.0,0.0,95.7,187.4,5.5,956.9,861.2,28,29.22
3,266.0,114.0,0.0,228.0,0.0,932.0,670.0,28,45.85
4,154.8,183.4,0.0,193.3,9.1,1047.4,696.7,28,18.29


In [6]:
import hackathon.src.datasets as datasets


X_train, X_test, y_train, y_test = datasets.cement_data(path + "/concrete.csv")

train_data, val_data = datasets.lightgbm_load_data(X_train, X_test, y_train, y_test)

In [9]:
import hackathon.src.surrogate as surrogate

model = surrogate.lightgbm_train(train_data, val_data)

rmse, mae, r2 = surrogate.lightgbm_evaluate(model, train_data, val_data)

print(f"RMSE: {rmse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")




Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 3.52072	valid's rmse: 4.7776
[200]	train's rmse: 2.64384	valid's rmse: 4.09028
[300]	train's rmse: 2.27059	valid's rmse: 3.82874
[400]	train's rmse: 2.0301	valid's rmse: 3.67862
[500]	train's rmse: 1.86138	valid's rmse: 3.59836
[600]	train's rmse: 1.75115	valid's rmse: 3.59284
Early stopping, best iteration is:
[636]	train's rmse: 1.72094	valid's rmse: 3.57101
RMSE: 3.5710, MAE: 2.4157, R²: 0.9530


In [139]:
from scipy.optimize import minimize

def inverse_function(y_target,model,init_x,method='BFGS'):
    def objective(x):
        x = x.reshape(1, -1)
        y_pred = surrogate.lightgbm_predict(model, x)
        # print(y_pred.shape)
        # print(y_target.shape)
        res = (y_pred - y_target)**2
        # print(res)
        return res
    result = minimize(objective, init_x, method=method)
    return result.x

In [140]:
surrogate.lightgbm_predict(model, X_test[0:1])

/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:722: UserWarning: Usage of np.ndarray subset (sliced data) is not recommended due to it will double the peak memory cost in LightGBM.
  _log_warning(


array([36.76990525])

In [141]:

x_inverse = inverse_function(y_test[0:1], model, X_train.mean(axis=0))

In [142]:
test = X_test[0:1]

In [143]:
from tqdm import tqdm

In [144]:
%%time
inverse_list = []
for sx,sy in tqdm(zip(X_test,y_test)):
    x_inverse = inverse_function(sy, model, X_train.mean(axis=0))
    inverse_list.append(x_inverse)


197it [00:37,  5.27it/s]

CPU times: user 4min 1s, sys: 16.2 s, total: 4min 18s
Wall time: 37.4 s


In [145]:
x_inverse = np.array(inverse_list)


In [146]:
SSE = np.sum((x_inverse - X_test)**2)
SST = np.sum((X_test - X_train.mean(axis=0))**2)
r2 = 1 - SSE/SST
r2

8.680197698285319e-06

In [147]:
%%time
inverse_list = []
for sx,sy in tqdm(zip(X_test,y_test)):
    x_inverse = inverse_function(sy, model, X_train.mean(axis=0),method='L-BFGS-B')
    inverse_list.append(x_inverse)

197it [00:10, 19.38it/s]

CPU times: user 1min 6s, sys: 4.26 s, total: 1min 10s
Wall time: 10.2 s


In [148]:
x_inverse = np.array(inverse_list)
SSE = np.sum((x_inverse - X_test)**2)
SST = np.sum((X_test - X_train.mean(axis=0))**2)
r2 = 1 - SSE/SST
print(r2)

3.4996702213963005e-07


In [149]:
%%time
inverse_list = []
for sx,sy in tqdm(zip(X_test,y_test)):
    x_inverse = inverse_function(sy, model, X_train.mean(axis=0),method='SLSQP')
    inverse_list.append(x_inverse)

197it [00:01, 106.70it/s]

CPU times: user 12 s, sys: 761 ms, total: 12.8 s
Wall time: 1.85 s


In [150]:
x_inverse = np.array(inverse_list)
SSE = np.sum((x_inverse - X_test)**2)
SST = np.sum((X_test - X_train.mean(axis=0))**2)
r2 = 1 - SSE/SST
print(r2)

-786418676994.156


In [151]:
%%time
inverse_list = []
for sx,sy in tqdm(zip(X_test,y_test)):
    x_inverse = inverse_function(sy, model, X_train.mean(axis=0),method='trust-constr')
    inverse_list.append(x_inverse)
x_inverse = np.array(inverse_list)
SSE = np.sum((x_inverse - X_test)**2)
SST = np.sum((X_test - X_train.mean(axis=0))**2)
r2 = 1 - SSE/SST
print(r2)

197it [00:00, 217.84it/s]

0.0
CPU times: user 5.98 s, sys: 374 ms, total: 6.36 s
Wall time: 909 ms


In [152]:
%%time
inverse_list = []
for sx,sy in tqdm(zip(X_test,y_test)):
    x_inverse = inverse_function(sy, model, X_train.mean(axis=0),method='COBYLA')
    inverse_list.append(x_inverse)
x_inverse = np.array(inverse_list)
SSE = np.sum((x_inverse - X_test)**2)
SST = np.sum((X_test - X_train.mean(axis=0))**2)
r2 = 1 - SSE/SST
print(r2)

197it [00:05, 37.63it/s]

-0.3491804360569728
CPU times: user 32.6 s, sys: 2.29 s, total: 34.9 s
Wall time: 5.24 s


In [153]:
%%time
inverse_list = []
for sx,sy in tqdm(zip(X_test,y_test)):
    x_inverse = inverse_function(sy, model, X_train.mean(axis=0),method='TNC')
    inverse_list.append(x_inverse)
x_inverse = np.array(inverse_list)
SSE = np.sum((x_inverse - X_test)**2)
SST = np.sum((X_test - X_train.mean(axis=0))**2)
r2 = 1 - SSE/SST
print(r2)

197it [00:13, 14.77it/s]

9.208726003961942e-10
CPU times: user 1min 26s, sys: 5.07 s, total: 1min 31s
Wall time: 13.3 s


In [155]:
%%time
inverse_list = []
for sx,sy in tqdm(zip(X_test,y_test)):
    x_inverse = inverse_function(sy, model, X_train.mean(axis=0),method='Nelder-Mead')
    inverse_list.append(x_inverse)
x_inverse = np.array(inverse_list)
SSE = np.sum((x_inverse - X_test)**2)
SST = np.sum((X_test - X_train.mean(axis=0))**2)
r2 = 1 - SSE/SST
print(r2)

197it [00:00, 358.10it/s]

0.0
CPU times: user 3.53 s, sys: 252 ms, total: 3.78 s
Wall time: 553 ms


In [156]:
%%time
inverse_list = []
for sx,sy in tqdm(zip(X_test,y_test)):
    x_inverse = inverse_function(sy, model, X_train.mean(axis=0),method='trust-exact')
    inverse_list.append(x_inverse)
x_inverse = np.array(inverse_list)
SSE = np.sum((x_inverse - X_test)**2)
SST = np.sum((X_test - X_train.mean(axis=0))**2)
r2 = 1 - SSE/SST
print(r2)

0it [00:00, ?it/s]


ValueError: Jacobian is required for trust region exact minimization.